## Notes
- Ensemble voting method for cluster labels?

## Bugs

## Combinations
- {lr: 0.0001, N: 200, niter: 20, n_iter: 300} 

### Import Packages

In [1]:
from pointnet import PointNet
from permnet import PermNet

import numpy as np
import time

import pyro
from pyro.distributions import *
from pyro.infer import Predictive, SVI, Trace_ELBO
from pyro.optim import Adam, AdamW
from pyro.nn import PyroModule

import torch
import torch.nn as nn
from torchvision import transforms, datasets

import matplotlib.pyplot as plt
%matplotlib inline

from tabulate import tabulate

### Load Data

In [2]:
# from https://medium.com/ai-society/gans-from-scratch-1-a-deep-introduction-with-code-in-pytorch-and-tensorflow-cb03cdcdba0f

def mnist_data():
    compose = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((.5), (.5))
        ])
    out_dir = './data/mnist'
    return datasets.MNIST(root=out_dir, train=True, transform=compose, download=True)

def images_to_vectors(images):
    return images.view(images.size(0), 784)

def vectors_to_images(vectors):
    return vectors.view(vectors.size(0), 1, 28, 28)

In [3]:
# Load data
N = 100
data = mnist_data()
# Create loader with data, so that we can iterate over it
data_loader = torch.utils.data.DataLoader(data, batch_size=N, shuffle=True)
# Num batches
num_batches = len(data_loader)

### Pyro

In [4]:
def model(data, step, assignment_list):
    # global variables
    alpha = torch.ones(T)
    weights = pyro.sample('weights', Dirichlet(alpha))
    
    with pyro.plate('components', T):
        locs = pyro.sample('locs', MultivariateNormal(torch.zeros(M), torch.eye(M)))

    # use PCA to reduce dimensionality so I can train faster
    img_set = images_to_vectors(data.squeeze())
    (U, S, V) = torch.pca_lowrank(img_set, q=None, center=True, niter=20)
    z = torch.matmul(data, V[:, :100000])
    # local variables
    with pyro.plate('data', N):
        assignment = pyro.sample('assignments', Categorical(weights))
        pyro.sample('obs', MultivariateNormal(locs[assignment], torch.eye(M)), obs=z)
        
def guide(data, step, assignment_list):
    # train nn if doing offline training
    if not amortize:
        pyro.module('alpha_mlp', alpha_mlp)
        pyro.module('tau_mlp', tau_mlp)
    
    if use_gpu: 
        data = data.cuda()
        
    tau = tau_mlp(data.float())
    tau = tau.view(T,M)

    # sample mixture components mu
    with pyro.plate('components', T):
        locs = pyro.sample('locs', MultivariateNormal(tau, torch.eye(M)))
    
    # sample cluster assignments
    alpha = alpha_mlp(data.float()) # returns a vector of length T
    weights = pyro.sample('weights', Dirichlet(alpha))  # vector of length T
    with pyro.plate('data', size=N):
        assignments = pyro.sample('assignments', Categorical(weights))
    
    # logging
    if step % log_iter == 0:
        
        print('='*10, 'Iteration {}'.format(step), '='*10)
        assignment_list.append(assignments)
        
        weight_data = [weights.squeeze()[i] for i in range(len(weights.squeeze()))]
        weight_data.insert(0, 'props')

        pc1_data = [locs[i,0] for i in range(locs.shape[0])]
        pc1_data.insert(0, 'pc1')

        pc2_data = [locs[i,1] for i in range(locs.shape[0])]
        pc2_data.insert(0, 'pc2')
        
        pc3_data = [locs[i,2] for i in range(locs.shape[0])]
        pc3_data.insert(0, 'pc3')
        
        pc4_data = [locs[i,3] for i in range(locs.shape[0])]
        pc4_data.insert(0, 'pc4')
        
        pc5_data = [locs[i,4] for i in range(locs.shape[0])]
        pc5_data.insert(0, 'pc5')
        
        pc6_data = [locs[i,5] for i in range(locs.shape[0])]
        pc6_data.insert(0, 'pc6')
        
        
        data = [weight_data, pc1_data, pc2_data, pc3_data, pc4_data, pc5_data, pc6_data]
        
        print(tabulate(data, headers=['', 'clust1', 'clust2', 'clust3', 'clust4', 'clust5', 'clust6', 'clust7', 'clust8', 'clust9', 'clust10']))

### Initialization

In [5]:
T = 10  # num components
M = 6  # how many features after pca

mlp_type = 'convnet'
amortize = False

if mlp_type == 'pointnet':
    alpha_mlp = PointNet(in_ch=784, output_size=T).float() 
    tau_mlp = PointNet(in_ch=784, output_size=T*M, softmax=False).float()
if mlp_type == 'permnet':
    alpha_mlp = PermNet(in_ch=784, hidden=2048, output_size=T).float()
    tau_mlp = PermNet(in_ch=784, hidden=2048, output_size=T*M).float()
if mlp_type == 'convnet':
    alpha_mlp = PermNet(in_ch=784, hidden=1024, output_size=T).float()
    tau_mlp = PermNet(in_ch=784, hidden=1024, output_size=T*M).float()
    
    
if amortize:
    saved_alpha_mlp = torch.load('saved_models/{}/mnist_alpha_mlp.pth'.format(mlp_type))
    saved_tau_mlp = torch.load('saved_models/{}/mnist_tau_mlp.pth'.format(mlp_type))
    
    alpha_mlp.load_state_dict(saved_alpha_mlp['model_state_dict'])
    tau_mlp.load_state_dict(saved_tau_mlp['model_state_dict'])

adam_params = {"lr": 0.001}
optimizer = Adam(adam_params)
svi = SVI(model, guide, optimizer, loss=Trace_ELBO())

use_gpu = torch.cuda.is_available()
if use_gpu:
    print('using GPU!')
    alpha_mlp = alpha_mlp.cuda()
    tau_mlp = tau_mlp.cuda()
else:
    print('not using GPU!')

not using GPU!


### Training

In [ ]:
dry = False
log_iter = 100

for n_batch, (real_batch, labels) in enumerate(data_loader):
    data = images_to_vectors(real_batch).unsqueeze(0)
    assignment_list = []
    elbo_ests = []
    n_steps = 500
    start = time.time()

    for step in range(n_steps):
        elbo_est = svi.step(data, step, assignment_list)
        elbo_ests.append(elbo_est)
        if step % log_iter == 0:
            end = time.time()
            print('took', end-start, 'seconds')
            start = time.time()

    if not dry:
        torch.save({'model_state_dict': alpha_mlp.state_dict(),
                    'assignments': assignment_list[-1],
                    'elbo_ests': elbo_ests,
                    'steps': n_steps,
                    'images': data,
                    'labels': labels
                   }, 'saved_models/{0}/mnist_alpha_mlp_{1}.pth'.format(mlp_type, n_steps))

        torch.save({'model_state_dict': tau_mlp.state_dict(),
                    'assignments': assignment_list[-1],
                    'elbo_ests': elbo_ests,
                    'steps': n_steps,
                    'images': data,
                    'labels': labels
                   }, 'saved_models/{0}/mnist_tau_mlp_{1}.pth'.format(mlp_type, n_steps))

    plt.plot(np.array(elbo_ests))
    plt.title('ELBO Plot')
    plt.xlabel('Iterations')
    plt.ylabel('ELBO')
    plt.savefig('figs/elbo_plot_mnist_{0}_{1}.png'.format(mlp_type, n_steps))
    break